# Optimize an Electrode

In [ ]:
# recon_path = '/Users/cu135/Software_Local/StimPyPer/stim_pyper/matlab_utils/test_electrode.mat'
recon_path = '/path/to/your/reconstruction.mat'
target_path = '/path/to/your/target.nii.gz'
out_dir = '/path/to/save/location'

In [ ]:
from stim_pyper.processing_utils.run_optimizer import OptimizerProcessor
optimizer = OptimizerProcessor(electrode_data_path=recon_path, nifti_path=target_path, output_path=out_dir)
d = optimizer.run()

Love,

--Calvin